In [1]:

from utils.prompt import template
from utils.utils import process_pdfs_from_dataframe_faiss

import pandas as pd
import string


## Dense DB (Vector DB) 및 Sparse DB

test.csv에서 질문들에 매칭되는 소스(pdf) 정보가 주어진다. 

각 질문에 따른 소스에 접근을 하고 리트리버를 사용해도 되기 때문에 

각 질문에 따른 소스별 DB를 만들어주자

In [2]:
df = pd.read_csv('../open/test.csv')
base_directory = '../open' # Your Base Directory

df.head()

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?


In [3]:
source = list(df['Source'].unique())

# 알파벳 리스트
alphabet = list(string.ascii_uppercase)

In [4]:
# 딕셔너리로 파일명과 알파벳 매핑
file_mapping = {alphabet[i]: source[i] for i in range(len(source))}

reversed_file_mapping = {value: key for key, value in file_mapping.items()}

In [5]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1-dacon-Q8:latest",   # llama3.1-dacon-Q8:latest,  llama3.1
        "temperature": 0
    },
    # "intfloat/multilingual-e5-small", intfloat/multilingual-e5-base, intfloat/multilingual-e5-large
    # BAAI/bge-m3
    "embed_model": "intfloat/multilingual-e5-large",  
    
    # pymupdf4llm, fitz, PyPDFLoader, PyMuPDFLoader, UnstructuredPDFLoader, PDFPlumberLoader, PyPDFDirectoryLoader
    "pdf_loader": "pymupdf4llm", 
    
    "text_split":{
        'chunk_size': 512,
        'chunk_overlap': 32
    },
    
    "save_data_path": './data_pymupdf4llm_512_32',
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.5,0.5],   # bm25와 vector 가중치


}

In [6]:
# PDF별 FAISS db 생성 
# from config import config
process_pdfs_from_dataframe_faiss(config, df, base_directory, reversed_file_mapping)

가지고 있는 pdf파일들: ['./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf'
 './test_source/보건복지부_부모급여(영아수당) 지원.pdf'
 './test_source/보건복지부_노인장기요양보험 사업운영.pdf'
 './test_source/산업통상자원부_에너지바우처.pdf' './test_source/국토교통부_행복주택출자.pdf'
 './test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'
 './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf']


Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


c:\Users\kim_h\anaconda3\envs\final\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:  11%|█         | 1/9 [00:11<01:32, 11.56s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


...

Processing PDFs:  78%|███████▊  | 7/9 [00:56<00:16,  8.30s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [01:04<00:08,  8.39s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [01:12<00:00,  8.03s/it]


In [9]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1-dacon-Q8:latest",   # llama3.1-dacon-Q8:latest,  llama3.1
        "temperature": 0
    },
    # "intfloat/multilingual-e5-small", intfloat/multilingual-e5-base, intfloat/multilingual-e5-large
    # BAAI/bge-m3
    "embed_model": "intfloat/multilingual-e5-large",  
    
    # pymupdf4llm, fitz, PyPDFLoader, PyMuPDFLoader, UnstructuredPDFLoader, PDFPlumberLoader, PyPDFDirectoryLoader
    "pdf_loader": "fitz", 
    
    "text_split":{
        'chunk_size': 512,
        'chunk_overlap': 32
    },
    
    "save_data_path": './data_fitz_512_32',
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.5,0.5],   # bm25와 vector 가중치


}

# PDF별 FAISS db 생성 
# from config import config
process_pdfs_from_dataframe_faiss(config, df, base_directory, reversed_file_mapping)

가지고 있는 pdf파일들: ['./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf'
 './test_source/보건복지부_부모급여(영아수당) 지원.pdf'
 './test_source/보건복지부_노인장기요양보험 사업운영.pdf'
 './test_source/산업통상자원부_에너지바우처.pdf' './test_source/국토교통부_행복주택출자.pdf'
 './test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'
 './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf']


Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:04<00:33,  4.20s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


...

Processing PDFs:  78%|███████▊  | 7/9 [00:33<00:10,  5.15s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:39<00:05,  5.23s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:44<00:00,  4.93s/it]


In [10]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1-dacon-Q8:latest",   # llama3.1-dacon-Q8:latest,  llama3.1
        "temperature": 0
    },
    # "intfloat/multilingual-e5-small", intfloat/multilingual-e5-base, intfloat/multilingual-e5-large
    # BAAI/bge-m3
    "embed_model": "intfloat/multilingual-e5-large",  
    
    # pymupdf4llm, fitz, PyPDFLoader, PyMuPDFLoader, UnstructuredPDFLoader, PDFPlumberLoader, PyPDFDirectoryLoader
    "pdf_loader": "PyPDFLoader", 
    
    "text_split":{
        'chunk_size': 512,
        'chunk_overlap': 32
    },
    
    "save_data_path": './data_PyPDFLoader_512_32',
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.5,0.5],   # bm25와 vector 가중치


}

# PDF별 FAISS db 생성 
# from config import config
process_pdfs_from_dataframe_faiss(config, df, base_directory, reversed_file_mapping)

가지고 있는 pdf파일들: ['./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf'
 './test_source/보건복지부_부모급여(영아수당) 지원.pdf'
 './test_source/보건복지부_노인장기요양보험 사업운영.pdf'
 './test_source/산업통상자원부_에너지바우처.pdf' './test_source/국토교통부_행복주택출자.pdf'
 './test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'
 './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf']


Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:05<00:40,  5.04s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


...

Processing PDFs:  78%|███████▊  | 7/9 [00:38<00:11,  5.70s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:43<00:05,  5.63s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:49<00:00,  5.49s/it]


In [11]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1-dacon-Q8:latest",   # llama3.1-dacon-Q8:latest,  llama3.1
        "temperature": 0
    },
    # "intfloat/multilingual-e5-small", intfloat/multilingual-e5-base, intfloat/multilingual-e5-large
    # BAAI/bge-m3
    "embed_model": "intfloat/multilingual-e5-large",  
    
    # pymupdf4llm, fitz, PyPDFLoader, PyMuPDFLoader, UnstructuredPDFLoader, PDFPlumberLoader, PyPDFDirectoryLoader
    "pdf_loader": "PyMuPDFLoader", 
    
    "text_split":{
        'chunk_size': 512,
        'chunk_overlap': 32
    },
    
    "save_data_path": './data_PyMuPDFLoader_512_32',
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.5,0.5],   # bm25와 vector 가중치


}

# PDF별 FAISS db 생성 
# from config import config
process_pdfs_from_dataframe_faiss(config, df, base_directory, reversed_file_mapping)

가지고 있는 pdf파일들: ['./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf'
 './test_source/보건복지부_부모급여(영아수당) 지원.pdf'
 './test_source/보건복지부_노인장기요양보험 사업운영.pdf'
 './test_source/산업통상자원부_에너지바우처.pdf' './test_source/국토교통부_행복주택출자.pdf'
 './test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'
 './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf']


Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:04<00:33,  4.23s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


...

Processing PDFs:  78%|███████▊  | 7/9 [00:33<00:09,  4.95s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:37<00:04,  4.93s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:42<00:00,  4.77s/it]


In [ ]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1-dacon-Q8:latest",   # llama3.1-dacon-Q8:latest,  llama3.1
        "temperature": 0
    },
    # "intfloat/multilingual-e5-small", intfloat/multilingual-e5-base, intfloat/multilingual-e5-large
    # BAAI/bge-m3
    "embed_model": "intfloat/multilingual-e5-large",  
    
    # pymupdf4llm, fitz, PyPDFLoader, PyMuPDFLoader, UnstructuredPDFLoader, PDFPlumberLoader, PyPDFDirectoryLoader
    "pdf_loader": "PDFPlumberLoader", 
    
    "text_split":{
        'chunk_size': 512,
        'chunk_overlap': 32
    },
    
    "save_data_path": './data_PDFPlumberLoader_512_32',
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.5,0.5],   # bm25와 vector 가중치


}

# PDF별 FAISS db 생성 
# from config import config
process_pdfs_from_dataframe_faiss(config, df, base_directory, reversed_file_mapping)

가지고 있는 pdf파일들: ['./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf'
 './test_source/보건복지부_부모급여(영아수당) 지원.pdf'
 './test_source/보건복지부_노인장기요양보험 사업운영.pdf'
 './test_source/산업통상자원부_에너지바우처.pdf' './test_source/국토교통부_행복주택출자.pdf'
 './test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'
 './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf']


Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:05<00:42,  5.32s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


...

Processing PDFs:  78%|███████▊  | 7/9 [00:44<00:13,  6.86s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:52<00:06,  6.99s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:58<00:00,  6.50s/it]
